In [11]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

train_images_dir = '/kaggle/input/deepfashion-1/datasets/train_images/'
test_images_dir = '/kaggle/input/deepfashion-1/datasets/test_images/'  
segm_dir = '/kaggle/input/deepfashion-1/datasets/segm/'  


train_image_paths = []
train_mask_paths = []
test_image_paths = []
test_mask_paths = []

for img_file in os.listdir(train_images_dir):
    if img_file.lower().endswith(('.png', '.jpg', '.jpeg')): 
        mask_file = img_file.replace('.png', '_segm.png').replace('.jpg', '_segm.png')  
        

        img_path = os.path.join(train_images_dir, img_file)
        mask_path = os.path.join(segm_dir, mask_file)
        
        if os.path.exists(mask_path):  
            train_image_paths.append(img_path)
            train_mask_paths.append(mask_path)

# Répéter pour le dossier de test
for img_file in os.listdir(test_images_dir):
    if img_file.lower().endswith(('.png', '.jpg', '.jpeg')): 
        mask_file = img_file.replace('.png', '_segm.png').replace('.jpg', '_segm.png')
        
        img_path = os.path.join(test_images_dir, img_file)
        mask_path = os.path.join(segm_dir, mask_file)
        
        if os.path.exists(mask_path)
            test_image_paths.append(img_path)
            test_mask_paths.append(mask_path)


for i in range(5):
    print(f"Image : {train_image_paths[i]} -> Masque : {train_mask_paths[i]}")


Image : /kaggle/input/deepfashion-1/datasets/train_images/WOMEN-Sweaters-id_00005253-06_1_front.png -> Masque : /kaggle/input/deepfashion-1/datasets/segm/WOMEN-Sweaters-id_00005253-06_1_front_segm.png
Image : /kaggle/input/deepfashion-1/datasets/train_images/WOMEN-Blouses_Shirts-id_00004602-01_4_full.png -> Masque : /kaggle/input/deepfashion-1/datasets/segm/WOMEN-Blouses_Shirts-id_00004602-01_4_full_segm.png
Image : /kaggle/input/deepfashion-1/datasets/train_images/WOMEN-Blouses_Shirts-id_00007196-04_4_full.png -> Masque : /kaggle/input/deepfashion-1/datasets/segm/WOMEN-Blouses_Shirts-id_00007196-04_4_full_segm.png
Image : /kaggle/input/deepfashion-1/datasets/train_images/MEN-Tees_Tanks-id_00004170-05_4_full.png -> Masque : /kaggle/input/deepfashion-1/datasets/segm/MEN-Tees_Tanks-id_00004170-05_4_full_segm.png
Image : /kaggle/input/deepfashion-1/datasets/train_images/WOMEN-Tees_Tanks-id_00003917-73_4_full.png -> Masque : /kaggle/input/deepfashion-1/datasets/segm/WOMEN-Tees_Tanks-id_000

In [12]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array


def load_and_preprocess_images(image_paths, mask_paths, target_size=(128, 128)):
    images = []
    masks = []
    
    for img_path, mask_path in zip(image_paths, mask_paths):
        # Charger l'image
        img = load_img(img_path, target_size=target_size)
        img_array = img_to_array(img) / 255.0  # Normaliser l'image (valeurs entre 0 et 1)
        images.append(img_array)
        
        # Charger le masque
        mask = load_img(mask_path, target_size=target_size, color_mode="grayscale")
        mask_array = img_to_array(mask) / 255.0  # Normaliser le masque (valeurs entre 0 et 1)
        masks.append(mask_array)
    
    # Convertir en arrays NumPy
    images = np.array(images)
    masks = np.array(masks)
    
    return images, masks

# Charger et prétraiter les images et les masques d'entraînement et de test
train_images, train_masks = load_and_preprocess_images(train_image_paths, train_mask_paths)
test_images, test_masks = load_and_preprocess_images(test_image_paths, test_mask_paths)

# Afficher les dimensions des données prétraitées
print(f"Train images shape: {train_images.shape}")
print(f"Train masks shape: {train_masks.shape}")


Train images shape: (10335, 128, 128, 3)
Train masks shape: (10335, 128, 128, 1)


In [3]:
import tensorflow as tf
from tensorflow.keras import layers

def unet_model(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    
    
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)
    
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)
    
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)
    

    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    

    u1 = layers.Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same')(c4)
    u1 = layers.concatenate([u1, c3])
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u1)
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c5)
    
    u2 = layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same')(c5)
    u2 = layers.concatenate([u2, c2])
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u2)
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c6)
    
    u3 = layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(c6)
    u3 = layers.concatenate([u3, c1])
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u3)
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c7)
    
    
    outputs = layers.Conv2D(num_classes, (1, 1), activation='softmax')(c7)
    
    model = tf.keras.Model(inputs, outputs)
    return model


input_shape = (128, 128, 3)
num_classes = 13 
model = unet_model(input_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_30 (Conv2D)  │ (None, 128, 128,  │      1,792 │ input_layer_2[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_31 (Conv2D)  │ (None, 128, 128,  │     36,928 │ conv2d_30[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_6     │ (None, 64, 64,    │          0 │ conv2d_31[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_32 (Conv2D)  │ (None, 64, 64,    │     73,856 │ max_pooling2d_6[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_33 (Conv2D)  │ (None, 64, 64,    │    147,584 │ conv2d_32[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_7     │ (None, 32, 32,    │          0 │ conv2d_33[0][0]   │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_34 (Conv2D)  │ (None, 32, 32,    │    295,168 │ max_pooling2d_7[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_35 (Conv2D)  │ (None, 32, 32,    │    590,080 │ conv2d_34[0][0]   │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_8     │ (None, 16, 16,    │          0 │ conv2d_35[0][0]   │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_36 (Conv2D)  │ (None, 16, 16,    │  1,180,160 │ max_pooling2d_8[… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_37 (Conv2D)  │ (None, 16, 16,    │  2,359,808 │ conv2d_36[0][0]   │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_6  │ (None, 32, 32,    │  1,179,904 │ conv2d_37[0][0]   │
│ (Conv2DTranspose)   │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_6       │ (None, 32, 32,    │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 512)              │            │ conv2d_35[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_38 (Conv2D)  │ (None, 32, 32,    │  1,179,904 │ concatenate_6[0]… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_39 (Conv2D)  │ (None, 32, 32,    │    590,080 │ conv2d_38[0][0]   │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_7  │ (None, 64, 64,    │    295,040 │ conv2d_39[0][0] 

 Total params: 8,558,285 (32.65 MB)

 Trainable params: 8,558,285 (32.65 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
from tensorflow.keras.callbacks import EarlyStopping
# Définir le callback EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',   
    patience=5,           # Nombre d'epochs sans amélioration après lesquelles l'entraînement s'arrête
    restore_best_weights=True  # Restaurer les meilleurs poids après l'arrêt
)

# Entraîner le modèle avec le callback
history = model.fit(
    train_images, train_masks,
    epochs=50,            # Nombre maximum d'epochs
    batch_size=32,
    validation_split=0.2,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint('unet_model.keras', monitor='val_loss', save_best_only=True),
        early_stopping    # Ajouter le callback EarlyStopping ici
    ]
)


import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Perte entraînement')
plt.plot(history.history['val_loss'], label='Perte validation')
plt.legend()
plt.show()


NameError: name 'train_images' is not defined